In [48]:
import numpy as np
from ase.io import read, write
from ase.neighborlist import NeighborList, NewPrimitiveNeighborList
import sys

In [25]:
# Initialize Julia
import julia
julia.install()

[ Info: Julia version info


Julia Version 1.9.2
Commit e4ee485e909 (2023-07-05 09:39 UTC)
Platform Info:
  OS: macOS (arm64-apple-darwin22.4.0)
  uname: Darwin 22.4.0 Darwin Kernel Version 22.4.0: Mon Mar  6 20:59:58 PST 2023; root:xnu-8796.101.5~3/RELEASE_ARM64_T6020 arm64 arm
  CPU: Apple M2 Pro: 
                 speed         user         nice          sys         idle          irq
       #1-10  2400 MHz    2651343 s          0 s    1716497 s   21067293 s          0 s
  Memory: 16.0 GB (59.40625 MB free)
  Uptime: 1.236531e6 sec
  Load Avg:  5.0009765625  6.79736328125  5.11572265625
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, apple-m1)
  Threads: 1 on 6 virtual cores
Environment:
  CFLAGS = -I/opt/homebrew/opt/openssl/include
  HOME = /Users/joemorrow
  HOMEBREW_CELLAR = /opt/homebrew/Cellar
  HOMEBREW_PREFIX = /opt/homebrew
  HOMEBREW_REPOSITORY = /opt/homebrew
  INFOPATH = /opt/homebrew/share/info:
  LDFLAGS = -L/opt/homebrew/opt/openssl/lib
  MANPATH = /opt/homebrew/share/man::
  P

[ Info: Julia executable: /Applications/Julia-1.9.app/Contents/Resources/julia/bin/julia
[ Info: Trying to import PyCall...
┌ Info: PyCall is already installed and compatible with Python executable.
│ 
│ PyCall:
│     python: /Users/joemorrow/miniconda3/envs/rings/bin/python
│     libpython: /Users/joemorrow/miniconda3/envs/rings/lib/libpython3.11.dylib
│ Python:
│     python: /Users/joemorrow/miniconda3/envs/rings/bin/python
└     libpython: 


In [39]:
# Create a Julia instance
jl = julia.Julia(compiled_modules=False)

In [46]:
import julia.Main as Main
Main.include("rings.jl")
Main.include("IOPiper.jl")

<PyCall.jlwrap Main.IOPiper>

In [49]:
def make_receiver(io):
            def receiver(s):
                io.write(s)
                io.flush()
            return receiver

Main.IOPiper.pipe_std_outputs(
    make_receiver(sys.stdout),
    make_receiver(sys.stderr))

Main.IOPiper.io_test()

Testing redirect


# ASE way of generating neighbourlist

In [97]:
ats = read('varMQ_11Ks_4096_500K_min.xyz', '-1')

In [32]:
ats = read('aSi_500atom_11Ks_annealed.xyz', '-1')

In [ ]:
ats = read("")

In [99]:
nl = NeighborList(cutoffs=2.85, self_interaction=False, bothways=True,
                  primitive=NewPrimitiveNeighborList)
nl.update(ats)

True

In [100]:
neighs = [nl.get_neighbors(i)[0] for i in range(len(ats))]

In [101]:
import csv

# Write the data to a file
with open('nodlnkd.txt', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',')
    for row in neighs:
        csvwriter.writerow(row+1)

In [105]:
len(ats)

4096

In [102]:
# select reference nodes
# pick from the corners of a tetrahedron to the edges of the cell, but inwards 20% of the lattice parameter

vecs = np.array([
    [0.5, 0.5, 0.5],
    [-0.5, -0.5, -0.5],
    [0.5, -0.5, 0.5],
    [0.5, 0.5, -0.5],
    [0., 0., 0.],
])*0.8 + np.ones(3)*0.5


In [103]:
spos = ats.get_scaled_positions()
refnodes = []

for v in vecs:
    diff = spos - v
    ref = np.argmin(np.linalg.norm(diff, axis=1))
    refnodes.append(ref+1) # Julia indexing

In [104]:
refnodes

[3920, 206, 1958, 2368, 1206]

In [69]:
rs = Main.rings.ring_statistics(len(ats), neighs, refnodes)

RuntimeError: <PyCall.jlwrap (in a Julia function called from Python)
JULIA: MethodError: no method matching (::Colon)(::Int64, ::Tuple{})

Closest candidates are:
  (::Colon)(::T, ::Any, !Matched::T) where T<:Real
   @ Base range.jl:45
  (::Colon)(::A, ::Any, !Matched::C) where {A<:Real, C<:Real}
   @ Base range.jl:10
  (::Colon)(::T, ::Any, !Matched::T) where T
   @ Base range.jl:44
  ...

Stacktrace:
 [1] ring_statistics(numatoms::Int64, nodlnkd::Vector{Vector{Int64}}, maxlvl::Int64, mxnodes::Int64, mxpths::Int64)
   @ Main.rings ~/data/julia_rings/rings.jl:208
 [2] ring_statistics(numatoms::Int64, nodlnkd::Vector{Vector{Int64}})
   @ Main.rings ~/data/julia_rings/rings.jl:208
 [3] invokelatest(::Any, ::Any, ::Vararg{Any}; kwargs::Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}})
   @ Base ./essentials.jl:816
 [4] invokelatest(::Any, ::Any, ::Vararg{Any})
   @ Base ./essentials.jl:813
 [5] _pyjlwrap_call(f::Function, args_::Ptr{PyCall.PyObject_struct}, kw_::Ptr{PyCall.P

In [33]:
nodlnkd

array([[  1,   8],
       [  1,   1],
       [  1,   3],
       ...,
       [512, 236],
       [512, 244],
       [512, 505]])

In [22]:
result = jl.eval("rings.ring_statistics()")

JuliaError: Exception 'MethodError: no method matching ring_statistics()

Closest candidates are:
  ring_statistics(!Matched::Any, !Matched::Any)
   @ Main.rings ~/data/julia_rings/rings.jl:205
  ring_statistics(!Matched::Any, !Matched::Any, !Matched::Any)
   @ Main.rings ~/data/julia_rings/rings.jl:205
  ring_statistics(!Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any)
   @ Main.rings ~/data/julia_rings/rings.jl:205
  ...
' occurred while calling julia code:
rings.ring_statistics()